# Setting up and importing data

In [1]:
import tensorflow as tf
import os
import zipfile

DESIRED_ACCURACY = 0.999

!wget --no-check-certificate \
    "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip" \
    -O "/tmp/happy-or-sad.zip"

zip_ref = zipfile.ZipFile("/tmp/happy-or-sad.zip", 'r')
zip_ref.extractall("/tmp/h-or-s")
zip_ref.close()

--2019-05-25 23:01:57--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.141.128, 2607:f8b0:400c:c06::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.141.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2670333 (2.5M) [application/zip]
Saving to: ‘/tmp/happy-or-sad.zip’

/tmp/happy-or-sad.z 100%[===================>]   2.55M  --.-KB/s    in 0.02s   

2019-05-25 23:01:57 (139 MB/s) - ‘/tmp/happy-or-sad.zip’ saved [2670333/2670333]



#Implementing a Callback

In [0]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('acc') > 0.999):
      print("\nReached 99.9% accuracy!")
      self.model.stop_training = True

callbacks = myCallback()

#Setting up the preprocessing ImageDataGenerator

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1/255)

train_generator = train_datagen.flow_from_directory(
        "/tmp/h-or-s",  
        target_size=(150, 150), 
        batch_size=10,
        class_mode='binary')

Found 80 images belonging to 2 classes.


#Define and Compile the Model

In [4]:
# This Code Block should Define and Compile the Model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation = 'relu'),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

from tensorflow.keras.optimizers import RMSprop

model.compile(optimizer = RMSprop(lr = 0.001),
             loss = 'binary_crossentropy',
             metrics = ['acc'])

Instructions for updating:
Colocations handled automatically by placer.


#Fit the Model

In [5]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=2,  
      epochs=15,
      verbose=1,
      callbacks=[callbacks])

Instructions for updating:
Use tf.cast instead.
Epoch 1/15
8/8 [==============================] - 3s 320ms/step - loss: 1.0559 - acc: 0.4625
Epoch 2/15
8/8 [==============================] - 2s 273ms/step - loss: 0.7112 - acc: 0.6125
Epoch 3/15
8/8 [==============================] - 2s 258ms/step - loss: 0.3965 - acc: 0.8500
Epoch 4/15
8/8 [==============================] - 2s 266ms/step - loss: 0.3592 - acc: 0.8500
Epoch 5/15
8/8 [==============================] - 2s 256ms/step - loss: 0.2467 - acc: 0.9125
Epoch 6/15
8/8 [==============================] - 2s 270ms/step - loss: 0.2144 - acc: 0.9000
Epoch 7/15
8/8 [==============================] - 2s 252ms/step - loss: 0.1902 - acc: 0.9000
Epoch 8/15
8/8 [==============================] - 2s 273ms/step - loss: 0.1018 - acc: 0.9750
Epoch 9/15
8/8 [==============================] - 2s 251ms/step - loss: 0.1058 - acc: 0.9750
Epoch 10/15
8/8 [==============================] - 2s 269ms/step - loss: 0.0483 - acc: 0.9875
Epoch 11/15
8/8 [====